In [4]:
import time
import board
from digitalio import DigitalInOut, Direction
import adafruit_st7789 as st7789
from PIL import Image, ImageDraw, ImageFont
import random

class JoystickPuzzle:
    def __init__(self):
        # Initialize display
        self.cs_pin = DigitalInOut(board.CE0)
        self.dc_pin = DigitalInOut(board.D25)
        self.reset_pin = DigitalInOut(board.D24)
        self.BAUDRATE = 24000000
        
        self.spi = board.SPI()
        self.disp = st7789.ST7789(
            self.spi,
            width=240,
            height=240,
            rotation=180,
            dc=self.dc_pin,
            rst=self.reset_pin,
            baudrate=self.BAUDRATE,
        )

        # Initialize joystick buttons
        self.button_U = self._setup_button(board.D17)
        self.button_D = self._setup_button(board.D22)
        self.button_L = self._setup_button(board.D27)
        self.button_R = self._setup_button(board.D23)
        self.button_A = self._setup_button(board.D5)
        self.button_B = self._setup_button(board.D6)

        # Setup display dimensions
        self.width = self.disp.width
        self.height = self.disp.height

        # Initialize game variables
        self.grid_size = 2
        self.cell_width = self.width // self.grid_size
        self.cell_height = self.height // self.grid_size
        self.puzzle = []
        self.blank_pos = (self.grid_size - 1, self.grid_size - 1)
        self.init_puzzle()

    def _setup_button(self, pin):
        button = DigitalInOut(pin)
        button.direction = Direction.INPUT
        return button

    def init_puzzle(self):
        """Initialize the puzzle grid."""
        self.puzzle = [[(j * self.grid_size + i + 1) for i in range(self.grid_size)] for j in range(self.grid_size)]
        self.puzzle[-1][-1] = 0  # Blank space
        self.shuffle_puzzle()

    def shuffle_puzzle(self):
        """Randomize the puzzle by simulating moves."""
        moves = ["up", "down", "left", "right"]
        for _ in range(100):
            move = random.choice(moves)
            self.move_blank(move)

    def move_blank(self, direction):
        """Move the blank tile in the specified direction."""
        x, y = self.blank_pos
        dx, dy = 0, 0
        if direction == "up":
            dy = -1
        elif direction == "down":
            dy = 1
        elif direction == "left":
            dx = -1
        elif direction == "right":
            dx = 1

        new_x, new_y = x + dx, y + dy
        if 0 <= new_x < self.grid_size and 0 <= new_y < self.grid_size:
            self.puzzle[y][x], self.puzzle[new_y][new_x] = self.puzzle[new_y][new_x], self.puzzle[y][x]
            self.blank_pos = (new_x, new_y)

    def render(self):
        """Render the puzzle to the display."""
        image = Image.new("RGB", (self.width, self.height))
        draw = ImageDraw.Draw(image)

        for j in range(self.grid_size):
            for i in range(self.grid_size):
                num = self.puzzle[j][i]
                x, y = i * self.cell_width, j * self.cell_height
                if num != 0:
                    draw.rectangle(
                        [x, y, x + self.cell_width, y + self.cell_height], fill=(255, 255, 255)
                    )
                    draw.text(
                        (x + self.cell_width // 2, y + self.cell_height // 2),
                        str(num),
                        fill=(0, 0, 0),
                        anchor="mm",
                    )
                else:
                    draw.rectangle(
                        [x, y, x + self.cell_width, y + self.cell_height], fill=(0, 0, 0)
                    )

        self.disp.image(image)

    def read_joystick(self):
        """Read joystick input and return a direction."""
        if not self.button_U.value:
            return "up"
        if not self.button_D.value:
            return "down"
        if not self.button_L.value:
            return "left"
        if not self.button_R.value:
            return "right"
        return None

    def is_solved(self):
        """Check if the puzzle is solved."""
        solved = [[(j * self.grid_size + i + 1) for i in range(self.grid_size)] for j in range(self.grid_size)]
        solved[-1][-1] = 0
        return self.puzzle == solved

    def run(self):
        """Main game loop."""
        while True:
            direction = self.read_joystick()
            if direction:
                self.move_blank(direction)

            if self.is_solved():
                print("Puzzle Solved!")
                break

            self.render()
            time.sleep(0.1)

# Run the game
game = JoystickPuzzle()
game.run()


RuntimeError: Too many displays